# K-Means and K-Medoids clustering from "Scratch"

The k-means clustering algorithm is one of the most basic forms of unsupervised
machine learning, and is foundational to a number of other ML methods. K-medoids
is a related variant with a very similar algorithm.

In this notebook I'll go over the general algorithms, the mathematical
intuitions for the methods, and implement simple "from scratch" functions in
python (i.e., a clear translation of the algo & math, but inefficient compared
to packaged implementations).

## The $k$-means clustering algorithm

The algorithm is really very straightforward:  
1. Initialize either with $k$ randomly selected points in the $n$-dimensional feature space or $k$ randomly selected data points
2. Assign all data points to the nearest of those randomly selected points
3. Find the centroid of those $k$ assigned clusters
4. Reassign all data points to the closest of those new $k$ centroids
5. Repeat steps 3 and 4 until the assignments and/or centroids stop changing.



In [1]:
import numpy as np